In [2]:
import numpy as np
import cv2
import os
import mediapipe as mp
import csv

In [3]:
def extract_hand_landmarks(image_path, hands):

    #Processing image 
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


    #Initializing mediapipe object and extracting results 
    results = hands.process(image_rgb)

    #Gathering required landmarks if exists
    both_hand_landmarks = []

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for landmark in hand_landmarks.landmark:
                # Extract x, y coordinates (relative to image dimensions)
                x = landmark.x
                y = landmark.y
                # Append coordinates to the list
                landmarks.append((x, y))
            both_hand_landmarks.append(landmarks)
        if len(both_hand_landmarks) == 1:
            both_hand_landmarks.append([(0, 0)] * len(both_hand_landmarks[0]))
    else:
        return None
    return both_hand_landmarks

In [5]:

# landmarks = ['class']
# for val in range(1, 22):
#     landmarks += ['lx{}'.format(val), 'ly{}'.format(val)]
# for val in range(1, 22):
#     landmarks += ['rx{}'.format(val), 'ry{}'.format(val)]
# len(landmarks)

85

In [27]:
# headers = landmarks
train_filename = "landmarks_train.csv"
test_filename = "landmarks_test.csv"
# with open(train_filename, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(headers)
# with open(test_filename, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(headers)

In [28]:
mp_hands = mp.solutions.hands
confidence = 0.7
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=confidence)


I0000 00:00:1709111551.557515   17408 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1709111551.558907   21027 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)


In [29]:

for gesture_dir in os.listdir('Dataset'):
    gesture_path = os.path.join('Dataset', gesture_dir)
    count = 0 
    print(f'Collecting landmarks for {gesture_dir}')
    for filename in os.listdir(gesture_path):
        if count >= 1000 :
            print('Collected 1000 images')
            break
        image_path = os.path.join(gesture_path, filename)
        results = extract_hand_landmarks(image_path, hands )
        if results:
            row = [gesture_dir]
            row += list(np.array(results).flatten())

        if count < 800:
            with open(train_filename, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(row)
            count += 1 
        else:
            with open(test_filename, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(row)
            count += 1 


/home/karthik/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
